In [1]:
import warnings
warnings.filterwarnings("ignore", message=r"Some weights of .* were not initialized from the model checkpoint", category=UserWarning,)
warnings.filterwarnings("ignore", message=r"Already found a `peft_config` attribute in the model", category=UserWarning,)
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from peft import PeftModel

BEST_MODEL_DIR = r"X:\Projects\Personal\Fake News Detector\outputs\checkpoints\BestModel"
tokenizer = DistilBertTokenizerFast.from_pretrained(BEST_MODEL_DIR)

base_model = DistilBertForSequenceClassification.from_pretrained(
    BEST_MODEL_DIR,
    num_labels=2,
    ignore_mismatched_sizes=True
)

model = PeftModel.from_pretrained(base_model, BEST_MODEL_DIR)
model.eval()
device = "cuda"
model.to(device)

def predict(texts):
    enc = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    with torch.no_grad():
        out = model(**enc)
        logits = out.logits

    probs = torch.softmax(logits, dim=-1).cpu().numpy()
    preds = probs.argmax(axis=-1)
    return preds, probs

if __name__ == "__main__":
    samples = [
        "Shashi Tharoor on Donald Trump's repeated ceasefire claims: ‘India didn’t need persuasion'",
    ]
    preds, probs = predict(samples)
    for txt, p, pr in zip(samples, preds, probs):
        label = "fake" if p==1 else "real"
        print(f"▶ INPUT: {txt}")
        print(f"  → Predicted: {label}  (p_real={pr[0]:.3f}, p_fake={pr[1]:.3f})\n")

▶ INPUT: Shashi Tharoor on Donald Trump's repeated ceasefire claims: ‘India didn’t need persuasion'
  → Predicted: fake  (p_real=0.001, p_fake=0.999)

